## Necessary Imports

In [2]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

## Mining

In [29]:
data = []
fn_dataf = pd.DataFrame(columns=['Country', 'Name', 'Rating', 'Review'])

### Getting Countries

In [58]:
df = pd.read_csv('world_population.csv')
countries = df['Country/Other']
countries= countries.iloc[183:]
countries

183                 Tunisia
184                  Turkey
185            Turkmenistan
186     U.S. Virgin Islands
187                  Uganda
188                 Ukraine
189    United Arab Emirates
190          United Kingdom
191           United States
192                 Uruguay
193              Uzbekistan
194                 Vanuatu
195               Venezuela
196                 Vietnam
197          Western Sahara
198                   Yemen
199                  Zambia
200                Zimbabwe
Name: Country/Other, dtype: object

In [60]:
for country in countries:
    browser = webdriver.Chrome('webdriver/chromedriver.exe')
    browser.get('http://google.com/')
    search_box = browser.find_element(By.NAME, 'q')
    search_box.send_keys(f'travel companies in {country}')
    search_box.submit()
    more_results = browser.find_element(By.XPATH, "//div[@class='MXl0lf tKtwEb wHYlTd']")
    more_results.click()
    all = BeautifulSoup(browser.page_source, 'html.parser')
    data.append(all.find_all('div', class_='rllt__details'))

    # Pagination
    page = all.find_all('a', class_='fl')
    while True:
        try:
            for i in range(len(page)):
                browser.find_element(By.LINK_TEXT, str(i+2)).click()
                time.sleep(10)
                go = BeautifulSoup(browser.page_source, 'html.parser')
                data.append(go.find_all('div', class_='rllt__details'))
        except:
            break
        break

    # Saving data
    data_n = []
    for n in range(len(data)):
        dets = BeautifulSoup(str(data[n]), 'html.parser')
        for (name, rate, review) in zip(dets.find_all('span', class_='OSrXXb'), dets.find_all('span', class_='YDIN4c YrbPuc'), dets.find_all('span', class_='HypWnf YrbPuc')):
            fn_dataf = fn_dataf.append({'Country': country, 'Name':name.get_text(), 'Rating': rate.get_text(), 'Review': review.get_text()}, ignore_index=True)
    browser.quit()

C:\Users\Shallom\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [61]:
fn_dataf

,Country,Name,Rating,Review
0,Slovakia,Best Slovakia Tours - Exclusive Private & Smal...,4.7,(10)
1,Slovakia,Travel agency Flora Tour,4.0,(22)
2,Slovakia,Slovakia Tours - sharing the real Slovakia wit...,4.5,(12)
3,Slovakia,"Tripito, cestovná kancelária - travel agency",4.8,(35)
4,Slovakia,Trip in Slovakia - Guided Tours & Excursions,5.0,(13)
...,...,...,...,...
86493,Zimbabwe,Rainbow Airlines,5.0,(2)
86494,Zimbabwe,Limpopo Valley Tourism Association - Musina,4.1,(11)
86495,Zimbabwe,Cell Insurance Company,4.3,(3)
86496,Zimbabwe,Impala Car Rental,4.4,(48)


In [63]:
sec = pd.read_csv('seventh_batch.csv')
sec.drop(columns=['Unnamed: 0'], inplace=True)
final = pd.concat([sec, fn_dataf], ignore_index=True)
final.to_csv('final_batch.csv')

In [66]:
df = pd.read_csv('datasets/final_batch.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,Country,Name,Rating,Review
0,Afghanistan,Sky Travel & Tours,4.3,(76)
1,Afghanistan,Air Gateway Tourist & Travel Agency,3.9,(55)
2,Afghanistan,Jamil Mangal Tourist & Travel Agency,4.3,(14)
3,Afghanistan,Tolo Travel & Tours,4.0,(23)
4,Afghanistan,Opal Travels,5.0,(9)
...,...,...,...,...
299652,Zimbabwe,Rainbow Airlines,5.0,(2)
299653,Zimbabwe,Limpopo Valley Tourism Association - Musina,4.1,(11)
299654,Zimbabwe,Cell Insurance Company,4.3,(3)
299655,Zimbabwe,Impala Car Rental,4.4,(48)


## Mining Financial Data

In [7]:
res = requests.get('https://opencorporates.com/companies?utf8=%E2%9C%93&q=Rainbow Airlines&commit=Go&jurisdiction_code=&utf8=%E2%9C%93&commit=Go&controller=searches&action=search_companies&inactive=false&mode=best_fields&search_fields%5B%5D=name&search_fields%5B%5D=previous_names&search_fields%5B%5D=company_number&search_fields%5B%5D=other_company_numbers&branch=&nonprofit=&order=')
fn = BeautifulSoup(res.content)
# inp = 
# inp
fn.find('p', {'id': 'no_results'})